<center><h1>TP n° 42 : Réponse d'un circuit RC</h1></center>

## Fonctionnement du TP

Présentation du TP. Faire une vrai description, et rappeler la formule de  :

$$\frac{d u_C}{d t} - \frac{1}{\tau} u_C = \frac{E}{\tau} \qquad ; \qquad \tau=RC$$

En vrai, il serait sympa de mettre une image du circuit.

Plusieurs bibliothèques sont disponibles (une seule importée au début) :
 - `module_communication_arduino`, module fait maison pour pouvoir **envoyer des commandes à l'Arduino depuis Python** ;
 - `TP_42` pour abréger des lignes de codes spécifiques à cette activité, qui n'apportent rien à l'élève (toutes cosmétique, la plupart pour les graphiques) ;
 - `time` pour les delais.

In [1]:
from module_communication_arduino import Arduino
import time

### Commandes disponibles

Trois commandes peuvent être envoyées à l'Arduino :
 - `GENERATEUR_ON`, qui allume le générateur ;
 - `GENERATEUR_OFF`, qui éteint ce dernier ;
 - `RELEVE_VALEUR`, qui demande de faire une mesure de tension $u_C$ et renvoie sa valeur par le port série ;

In [2]:
# Commandes Arduino disponibles
GENERATEUR_ON ="0"
GENERATEUR_OFF="1"
RELEVE_VALEUR ="2"

### Syntaxe Python
Pour cela, un objet `Arduino` permet de faire abstraction du côté technique. Pour envoyer une commande (et recevoir une mesure), on procède comme suit :

In [3]:
# Initialise la communication avec l'Arduino :
mon_arduino = Arduino("/dev/ttyACM0")
time.sleep(1) # Attendre 1 seconde

In [6]:
# Réalisation d'une mesure
message_retour = mon_arduino.envoi_message_et_recoit_reponse(RELEVE_VALEUR)
message_retour

'6928;361'

La valeur renvoyée par l'Arduino n'est pas exploitable telle quelle : il s'agit d'une chaine de caractères, composée des deux valeurs séparées d'un `;`. Il faut faire les transformations suivantes :
 - Séparer en deux variables la chaine de caractères ;
 - Convertir ces deux valeurs en `float` ;
 - Convertir l'unité de la première valeur (un temps exprimé en millisecondes) en secondes
 - Transformer la seconde valeur (un nombre de bit issu d'un CAN) en tension réelle (0 bit → 0 V, 1024 bit → 5V)

**Ces opérations tiennent plus de l'informatique que de la physique... Pas la peine de trop s'y attarder** : cette transformation existe dans la bibliothèque `TP_42` :

In [7]:
from TP_42 import transforme_message_arduino

t_act, Uc_act = transforme_message_arduino(message_retour)
print("Temps (s) : ", t_act)
print("Tension Uc (V) : ", Uc_act)

Temps (s) :  6.928
Tension Uc (V) :  1.7626953125


------

## Mesure automatique de la charge/décharge d'un condensateur

### 1) Charge du condensateur

Vu qu'on peut *piloter* l'Arduino, il ne reste plus qu'à transcrire ce que l'on veut faire.

Pour mesurer la charge du condensateur, on doit :
 1. Éteindre le générateur pour laisser le condensateur se décharger ;
 2. Attendre la décharge ;
 3. Allumer le générateur ;
 4. Mesurer la tension $u_C$ tout au long de la charge.
 
Pour cette dernière étape : **on réalise 500 mesures successives** . À chaque mesure, on sauvegarde le temps mesuré `t_act` dans une liste `t`, et la tension mesurée `Uc_act` dans une liste `Uc`.

Le code suivant réalise les étapes 1. , 2. , et 3. Comment réaliser l'étape 4 ?

In [8]:
t,Uc = [],[] # Sauvegarde des mesures
N=500        # Nombre de points de mesure

# Étape 1.
mon_arduino.envoi_message_et_recoit_reponse(GENERATEUR_OFF)

# Étape 2.
time.sleep(5) # On attends 5 secondes, pour que le condensateur se décharge

# Étape 3.
mon_arduino.envoi_message_et_recoit_reponse(GENERATEUR_ON)
time.sleep(1)

############################################# COMPLÉTER ICI LE CODE : #############################################
# Déjà rempli pour l'exemple : on écrit une simple boucle for.
# Si l'élève est malin (= sait se qu'est un copier-coller), il verra qu'il suffit de CC les cases précédentes
# et de les mettre dans une boucle for.

# Étape 4.
for i in range(N):
    message_retour = mon_arduino.envoi_message_et_recoit_reponse(RELEVE_VALEUR)
    t_act, Uc_act = transforme_message_arduino(message_retour)
    t.append(t_act)
    Uc.append(Uc_act)

###################################################################################################################

# On éteinds le générateur par sécurité 
mon_arduino.envoi_message_et_recoit_reponse(GENERATEUR_OFF)

# Transtypage utile pour la suite
import numpy as np
t=np.array(t)
Uc=np.array(Uc)

Pour finir, on affiche le résultat (avec un joli graphique, dont la cosmétique est géré par la fonction `fait_un_joli_graphique` ) :

In [12]:
from matplotlib import pyplot as plt
%matplotlib qt
# Affichage d'un graphique dans une fenêtre à part ;
# en cas de problème, remplacer 'qt' par 'notebook' (peut être plus lent)

from TP_42 import fait_un_joli_graphique, SnappingCursor

line, = plt.plot(t-t[0],Uc, label="Mesures")
fait_un_joli_graphique()

cursor = SnappingCursor(plt.gca(), line, useblit=True, color='black', alpha=0.5, ls="dashed")

plt.show()